In [195]:
#Data Structure 
tag_sp=["INDI", "FAM"]
#Level Zero Tags
tag_zero=["HEAD", "TRLR", "NOTE"]
#Level One Tags
tag_one=["NAME", "SEX", "BIRT", "DEAT","FAMC","FAMS","MARR", "DIV","HUSB","WIFE","CHIL"]
#Level Zero Tags
tag_fam={"INDI":["NAME", "SEX","BIRT", "DEAT","FAMC","FAMS"],
             "FAM":["MARR", "DIV","HUSB","WIFE","CHIL"], 
             "DATE":["BIRT", "DEAT", "DIV", "MARR"]}


In [196]:
def isDateParent(A):
    return A[1] in tag_fam["DATE"]

In [197]:
def month_to_num(shortMonth):
    return{
        'JAN' : "1",
        'FEB' : "2",
        'MAR' : "3",
        'APR' : "4",
        'MAY' : "5",
        'JUN' : "6",
        'JUL' : "7",
        'AUG' : "8",
        'SEP' : "9", 
        'OCT' : "10",
        'NOV' : "11",
        'DEC' : "12"
    }[shortMonth]

In [198]:
def convert_date(date_arr):
    return "{}-{}-{}".format(date_arr[2], month_to_num(date_arr[1]), date_arr[0])
    

In [199]:
from datetime import datetime

def determine_age(birth_date, death_date):
    if death_date:
        return int(death_date.split('-')[0]) - int(birth_date.split('-')[0])
    else:
        today = datetime.today()
        return today.year - int(birth_date.split('-')[0])

In [200]:
def find_name(arr, _id):
    #takes array of dict objects
    for indi in arr:
        if _id == indi["INDI"]:
            return indi["NAME"]

In [201]:
def read_in(file):
    doc={"INDI":[], "FAM":[]}
    dic={}
    flag=False #indicates whether the correct tag has appeared before DATE tag
    with open(file) as f:
        for line in f:
            A=line.strip().split(" ")
            #if the current tag is individual
            if len(A)==3 and A[0]=='0' and A[2]== "INDI":
                if dic:
                    if current_tag == 'INDI':
                        if 'DEAT' in dic:
                            age = determine_age(dic['BIRT'], dic['DEAT'])
                            alive = False
                        else:
                            age = determine_age(dic['BIRT'], None)
                            alive = True
                            dic['DEAT'] = 'NA'
                        dic["AGE"] = str(age)
                        dic['ALIVE']= alive
                    doc[current_tag].append(dic)
                current_tag="INDI"
                dic={}
                dic["INDI"]=A[1] #inserts individual's ID into the dictionary
            elif len(A)==3 and A[0]=='0' and A[2]=="FAM":
                if dic:
                    if current_tag == 'INDI':
                        if 'DEAT' in dic:
                            age = determine_age(dic['BIRT'], dic['DEAT'])
                            alive = False
                        else:
                            age = determine_age(dic['BIRT'], None)
                            alive = True
                            dic['DEAT'] = 'NA'
                        dic["AGE"] = str(age)
                        dic['ALIVE']= alive
                    doc[current_tag].append(dic) 
                current_tag = "FAM"
                dic = {}
                dic["FAM"] = A[1]
            elif (A[1]=="DATE" and flag):
                flag=False
                date_arr = A[2:] #extracts the date argument from the line
                dic[tmp]= convert_date(date_arr) #converts the date into correct format
            elif A[0]=='1' and A[1] in tag_one:
                if (isDateParent(A)): #determines whether the current tag is parent of DATE tag
                    tmp=A[1] #extracts the tag name
                    flag=True
                else: 
                    #current tag is not the parent tag of DATE tag
                    if A[1] == "HUSB":
                        husband=find_name(doc["INDI"], A[2])
                        dic["HUSB_NAME"]=husband
                    if A[1] == "WIFE":
                        husband=find_name(doc["INDI"], A[2])
                        dic["WIFE_NAME"]=husband
                    if A[1] == 'CHIL':
                        #INDI_CHILD indicates all the children within a family
                        children = dic["FAM_CHILD"] if "FAM_CHILD" in dic else []
                        children.append(A[2])
                        dic["FAM_CHILD"] = children
                    if A[1] == 'FAMC' or A[1] == 'FAMS':
                        child = dic["INDI_CHILD"] if "INDI_CHILD" in dic else []
                        spouse = dic["SPOUSE"] if "SPOUSE" in dic else []
                        if A[1] == 'FAMC':
                            child.append(A[2])
                        else:
                            spouse.append(A[2])
                        dic['INDI_CHILD'] = child #FAM_CHILD indicates which family this individual belongs to
                        dic['SPOUSE'] = spouse
                    else: #other type of tag
                        dic[A[1]]=' '.join(A[2:])
        return doc   
                  

In [202]:
document= read_in("./Yujie_Test.ged")

In [204]:
from prettytable import PrettyTable
for family in document["FAM"]:
    husband=family["HUSB"] if "HUSB" in family else []
    
    
indi_table = PrettyTable() #individual table
indi_table.field_names = ["ID", "Name", "Gender", "Birthday", "Age", "Alive", "Death", "Child", "Spouse"]
fam_table = PrettyTable() #family table
fam_table.field_names = ["ID", "Married", "Divorced", "Husband ID", "Husband Name", "Wife ID", "Wife Name", "Children"]
for individual in document["INDI"]:
    if not individual["SPOUSE"]:
        individual["SPOUSE"] = "NA"
    indi_table.add_row([individual["INDI"], individual["NAME"], individual["SEX"], individual["BIRT"], individual["AGE"], 
                       individual["ALIVE"], individual["DEAT"], ("".join(individual["INDI_CHILD"])), ("".join(individual["SPOUSE"]))])
    
for family in document["FAM"]:
    if "DIV" not in family:
        family["DIV"] = "NA"
    if "HUSB" not in family:
        family["HUSB"] = "NA"
    if "HUSB_NAME" not in family:
        family["HUSB_NAME"] = "NA"
    if "WIFE" not in family:
        family["WIFE"] = "NA"
    if "WIFE_NAME" not in family:
        family["WIFE_NAME"] = "NA"
    if "FAM_CHILD" not in family:
        family["FAM_CHILD"] = "NA"
    if "MARR" not in family:
        family["MARR"] = "NA"
        
    fam_table.add_row([family["FAM"], family["MARR"], family["DIV"], family["HUSB"], family["HUSB_NAME"], family["WIFE"], family["WIFE_NAME"], ("".join(family["FAM_CHILD"]))])
    
print(indi_table)
print(fam_table)

+-------+-------------------+--------+------------+-----+-------+----------+-------+----------+
|   ID  |        Name       | Gender |  Birthday  | Age | Alive |  Death   | Child |  Spouse  |
+-------+-------------------+--------+------------+-----+-------+----------+-------+----------+
|  @I1@ |   Jimmy /Colon/   |   M    |  1912-6-5  |  86 | False | 1998-5-2 |  @F2@ |   @F1@   |
|  @I2@ |   Helen /Colon/   |   F    | 1920-12-10 |  82 | False | 2002-6-2 |       |   @F1@   |
|  @I3@ |    Dora /Colon/   |   F    |  1951-4-5  |  68 |  True |    NA    |  @F1@ | @F3@@F4@ |
|  @I4@ |   Rurra /Colon/   |   F    |  1952-9-8  |  67 |  True |    NA    |  @F1@ |   @F5@   |
|  @I5@ |    John /Colon/   |   M    | 1957-5-11  |  62 |  True |    NA    |  @F1@ |   @F6@   |
|  @I6@ |  Micheal /Huston/ |   M    | 1950-5-10  |  69 |  True |    NA    |       |   @F3@   |
|  @I7@ |   Samul /Hilson/  |   M    |  1952-7-5  |  67 |  True |    NA    |       |   @F4@   |
|  @I8@ |   Henry /Nilson/  |   M    | 1